![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [78]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below
df =  pd.read_csv('rental_info.csv')

df['rental_date']= pd.to_datetime(df['rental_date'])
df['return_date']= pd.to_datetime(df['return_date'])
# print(df.info())

df['rental_length_days'] = (df['return_date'] - df['rental_date']).dt.days
# print(df[['rental_date','return_date','rental_length_days' ]])

print(df.columns)
print(df['special_features'].value_counts())
df['deleted_scenes']  =df['special_features'].apply(lambda x : 1 if 'Deleted Scenes' in x  else 0)
df['behind_the_scenes'] = df['special_features'].apply(lambda x : 1 if 'Behind the Scenes' in x  else 0)
print(df[['deleted_scenes','behind_the_scenes']])


Index(['rental_date', 'return_date', 'amount', 'release_year', 'rental_rate',
       'length', 'replacement_cost', 'special_features', 'NC-17', 'PG',
       'PG-13', 'R', 'amount_2', 'length_2', 'rental_rate_2',
       'rental_length_days'],
      dtype='object')
{Trailers,Commentaries,"Behind the Scenes"}                     1308
{Trailers}                                                      1139
{Trailers,Commentaries}                                         1129
{Trailers,"Behind the Scenes"}                                  1122
{"Behind the Scenes"}                                           1108
{Commentaries,"Deleted Scenes","Behind the Scenes"}             1101
{Commentaries}                                                  1089
{Commentaries,"Behind the Scenes"}                              1078
{Trailers,"Deleted Scenes"}                                     1047
{"Deleted Scenes","Behind the Scenes"}                          1035
{"Deleted Scenes"}                            

In [79]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

# Assuming df is already defined
X1 = df.drop(columns=['rental_length_days', 'return_date', 'rental_date', 'special_features'])
X = X1
y = df['rental_length_days']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

las = Lasso(alpha=0.2)
las.fit(X_train_scaled, y_train)
coef = las.coef_

# Convert numpy arrays back to DataFrame to use iloc
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

X_train1 = X_train_scaled_df.iloc[:, coef > 0]
X_test1 = X_test_scaled_df.iloc[:, coef > 0]

In [80]:
print(df.info)

<bound method DataFrame.info of                     rental_date  ... behind_the_scenes
0     2005-05-25 02:54:33+00:00  ...                 1
1     2005-06-15 23:19:16+00:00  ...                 1
2     2005-07-10 04:27:45+00:00  ...                 1
3     2005-07-31 12:06:41+00:00  ...                 1
4     2005-08-19 12:30:04+00:00  ...                 1
...                         ...  ...               ...
15856 2005-08-22 10:49:15+00:00  ...                 1
15857 2005-07-31 09:48:49+00:00  ...                 1
15858 2005-08-20 10:35:30+00:00  ...                 1
15859 2005-07-31 13:10:20+00:00  ...                 1
15860 2005-08-18 06:33:55+00:00  ...                 1

[15861 rows x 18 columns]>


In [81]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
lr = LinearRegression()
lr.fit(X_train1,y_train)
y_pred = lr.predict(X_test1)
mse_lr = mean_squared_error(y_test,y_pred)

param_grid = { 'max_depth': np.arange(1,200,1),
              'min_samples_leaf': np.arange(0,1,0.2),
              'max_depth': np.arange(1,20,1)
    
}
ran = RandomizedSearchCV(estimator=DecisionTreeRegressor(), param_distributions= param_grid, random_state= 9)
ran.fit(X_train1,y_train)
best_dt = ran.best_estimator_
best_dt.fit(X_train,y_train)
y_pred1 = best_dt.predict(X_test)
mse_dt = mean_squared_error(y_test,y_pred)

param_grid1 = {'n_estimators': np.arange(1,101,1),
          'max_depth':np.arange(1,11,1)
    
}
rad1 = RandomizedSearchCV(estimator=RandomForestRegressor(),param_distributions= param_grid1 , random_state = 9 )
rad1.fit(X_train1, y_train)
best_rf = rad1.best_estimator_
best_rf.fit(X_train,y_train)
y_pred_ra = best_rf.predict(X_test)

mse_rf = mean_squared_error(y_test,y_pred_ra)

best_model = RandomForestRegressor()
best_mse = mse_rf



In [82]:
print(mse_lr)
# print(lr.get_params())
# print(RandomForestRegressor().get_params())
print(best_dt, mse_dt)
print(best_rf , mse_rf)

4.845903109515722
DecisionTreeRegressor(max_depth=10, min_samples_leaf=0.2) 4.845903109515722
RandomForestRegressor(max_depth=6, n_estimators=78) 2.456541867411209
